# Recurrent Neural Network
Finds the tone of an article based on the article's headline, abstract and key words

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Bidirectional, LSTM, Dropout, Dense, Embedding, Flatten
from keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [2]:
# Load training data
train_df = pd.read_csv('../data/cleaned_train.csv')
x_train = np.array(train_df['numerical_sequence'].apply(lambda x: [int(i) for i in x.split(',')]).tolist())
y_train = np.array(train_df['BERT_sentiment_score'])

C:\Users\User\AppData\Local\Temp\ipykernel_11716\1412115869.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train = np.array(train_df['numerical_sequence'].apply(lambda x: [int(i) for i in x.split(',')]).tolist())


In [3]:
# Load test data
test_df = pd.read_csv('../data/cleaned_test.csv')
x_test = np.array(test_df['numerical_sequence'].apply(lambda x: [int(i) for i in x.split(',')]).tolist())
y_test = np.array(test_df['BERT_sentiment_score'])

C:\Users\User\AppData\Local\Temp\ipykernel_11716\4022124299.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_test = np.array(test_df['numerical_sequence'].apply(lambda x: [int(i) for i in x.split(',')]).tolist())


In [6]:
# Pad sequences to a fixed length
maxlen = 227
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen, truncating='post')
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen, truncating='post')

In [7]:
# Training and validation split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [8]:
vocab_size = max(np.max(x_train), np.max(x_test)) + 1

In [9]:
# Define the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', 
                               # Number of epochs to wait for improvement
                               patience=3,  
                               verbose=1, 
                               # Restore the weights of the best epoch
                               restore_best_weights=True)  

In [10]:
#Hyperparameters
num_classes = 3

In [11]:
# Define the LSTM model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=32, input_length=x_train.shape[1]))
model.add(Bidirectional(LSTM(units=50, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(units=50, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(units=50, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [12]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Build the model
model.build(x_train.shape)

In [13]:
# Train the model
model.fit(x_train, y_train, batch_size=16, epochs=20, validation_data=(x_val, y_val), 
         callbacks=[early_stopping])

Epoch 1/20
 15/623 [..............................] - ETA: 3:48 - loss: 1.0910 - accuracy: 0.4208

KeyboardInterrupt: 

In [25]:
# save the trained model
model.save('../models/RNN_classification_model.h5')

In [26]:
model = load_model('../models/RNN_classification_model.h5')

In [27]:
# Evaluate the model on validation data
loss, accuracy = model.evaluate(x_val, y_val)
print('Validation loss:', loss)
print('Validation accuracy:', accuracy)

63/63 [==============================] - 9s 102ms/step - loss: 0.8375 - accuracy: 0.6369
Validation loss: 0.8374772667884827
Validation accuracy: 0.6369107365608215
